In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Import packages & Load data

In [2]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple
import org.jetbrains.letsPlot.intern.Plot
import org.um.feri.ears.statistic.rating_system.true_skill.TrueSkillRating
import org.um.feri.ears.statistic.rating_system.true_skill.Team
import org.um.feri.ears.statistic.rating_system.GameInfo
import org.um.feri.ears.statistic.rating_system.Player
import org.um.feri.ears.statistic.rating_system.true_skill.TwoPlayerTrueSkillCalculator
import org.um.feri.ears.statistic.rating_system.glicko2.Glicko2RatingCalculator
import org.um.feri.ears.visualization.rating.RatingIntervalPlot
import org.um.feri.ears.statistic.rating_system.RatingType

import org.um.feri.ears.statistic.rating_system.GameResult
import org.um.feri.ears.statistic.rating_system.glicko2.Game
import kotlin.random.Random

In [ ]:
/*var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData: GPAlgorithmMultiConfigurationsProgressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)
*/

In [3]:
//val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor_Robostrike_Short_Add_SSOS.ser""";
//val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor_Gens.ser""";

//val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor_comparison=evaluations_reruns=11_pop=100_evals=20600.ser""";
//val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor_comparison=generations_reruns=11_pop=100_evals=3600.ser""";
val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor.ser""";

var gpAlgorithmExecutor: GPAlgorithmExecutor = GPAlgorithmExecutor.deserializeGPAlgorithmExecutorState(gpDataFile);
var progressData: GPAlgorithmMultiConfigurationsProgressData = gpAlgorithmExecutor.multiConfigurationsProgressData;

# Update Configuration parameters

In [ ]:
// Change configuration parameters
/*for (i in 0 until gpAlgorithmExecutor.configuration.Configurations.size) {
    var obj = gpAlgorithmExecutor.configuration.Configurations.get(i).UnityConfiguration
    if (obj is MutableMap<*, *>) {
        @Suppress("UNCHECKED_CAST")
        val map = obj as MutableMap<String, Any?>
        map["IndividualsSourceJSON"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\"
        map["IndividualsSourceSO"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\SOs\\"
        println("Updated IndividualsSourceJSON: ${map["IndividualsSourceJSON"]}")
        println("Updated Map: $map")
    } else {
        println("Object is not a MutableMap")
    }

    obj = gpAlgorithmExecutor.configuration.Configurations.get(i).UnityConfigurationConvergenceGraph
    if (obj is MutableMap<*, *>) {
        @Suppress("UNCHECKED_CAST")
        val map = obj as MutableMap<String, Any?>
        map["IndividualsSourceJSON"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\"
        map["IndividualsSourceSO"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\SOs\\"
        println("Updated IndividualsSourceJSON: ${map["IndividualsSourceJSON"]}")
        println("Updated Map: $map")
    } else {
        println("Object is not a MutableMap")
    }
    obj = gpAlgorithmExecutor.configuration.Configurations.get(i).UnityConfigurationMasterTournamentGraph
    if (obj is MutableMap<*, *>) {
        @Suppress("UNCHECKED_CAST")
        val map = obj as MutableMap<String, Any?>
        map["IndividualsSourceJSON"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\"
        map["IndividualsSourceSO"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\SOs\\"
        println("Updated IndividualsSourceJSON: ${map["IndividualsSourceJSON"]}")
        println("Updated Map: $map")
    } else {
        println("Object is not a MutableMap")
    }
}

var obj2 = gpAlgorithmExecutor.configuration.FinalMasterTournamentsConfiguration.UnityConfiguration
if (obj2 is MutableMap<*, *>) {
    @Suppress("UNCHECKED_CAST")
    val map = obj2 as MutableMap<String, Any?>
    map["IndividualsSourceJSON"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\"
    map["IndividualsSourceSO"] = "C:\\Users\\marko\\Documents\\marko\\GenIATraP_Setup_New\\Resources\\Soccer\\SOs\\"
    println("Updated IndividualsSourceJSON: ${map["IndividualsSourceJSON"]}")
    println("Updated Map: $map")
} else {
    println("Object is not a MutableMap")
}

GPAlgorithmExecutor.serializeGPAlgorithmExecutorState(gpAlgorithmExecutor, "C:\\Users\\marko\\Documents\\FERI\\Projekti\\EARS\\GPAlgorithmExecutor.ser")
 */

# Prepare helper classes & functions

In [4]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [5]:
class MethodScore(
    var methodName: String,
    var wins: Int,
    var draws: Int,
    var losses: Int,
    var player: Player
) {
    constructor(methodName: String) : this(methodName, 0, 0, 0, Player(methodName))
}

In [6]:
class MethodRating (
    var methodName: String,
    var rating: Double,
    var stdDeviation: Double
) {
    constructor(methodName: String) : this(methodName, 0.0, 0.0)
}


In [7]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult, sumAbsValues: Boolean = false): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        if(sumAbsValues){
            sum += Math.abs(individualMatchResult.individualValues[key]!!)
        }
        else {
            sum += individualMatchResult.individualValues[key]!!
        }
    }

    return sum
}

In [8]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, avgGenIndividualRatings: MutableList<Double>?, bestGenIndividualsRatingAvgFitness: MutableList<Double>?, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, avgGenIndividualAvgFitnesses: MutableList<Double>?){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.getPopulationSimple()[0]
        var worstIndividualRating = genProgressData.getPopulationSimple()[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.getPopulationSimple()[0]
        var worstIndividualAvgFitness = genProgressData.getPopulationSimple()[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.getPopulationSimple()) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        if (bestGenIndividualsRating != null) {
            bestGenIndividualsRating.add(bestGenIndividualRating)
        }
        if (worstGenIndividualsRating != null) {
            worstGenIndividualsRating.add(worstIndividualRating)
        }

        avgGenIndividualRating /= genProgressData.getPopulationSimple().size
        if (avgGenIndividualRatings != null) {
            avgGenIndividualRatings.add(avgGenIndividualRating)
        }

        if(bestGenIndividualsRatingAvgFitness != null){
            bestGenIndividualsRatingAvgFitness.add(sumIndividualValues(bestGenIndividualRating.finalIndividualFitness.avgMatchResult))
        }

        if (bestGenIndividualAvgFitnesses != null) {
            bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        }
        if (worstGenIndividualAvgFitnesses != null) {
            worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)
        }

        avgGenIndividualAvgFitness /= genProgressData.getPopulationSimple().size
        if (avgGenIndividualAvgFitnesses != null) {
            avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
        }
    }
}

In [9]:
// TODO Move this to GPProgramSolutionSimple
fun calculateDiversity(solution1: GPProgramSolutionSimple, solution2: GPProgramSolutionSimple, w1: Double, w2: Double) : Double {
    var nodeCountDiversity = 0.0

    // Gather all different node count key
    val nodeCountKeys = mutableSetOf<String>()
    nodeCountKeys.addAll(solution1.nodeCounts.keys)
    nodeCountKeys.addAll(solution2.nodeCounts.keys)

    // Calculate node count diversity
    for (key in nodeCountKeys) {
        val nodeCount1 = solution1.nodeCounts[key] ?: 0
        val nodeCount2 = solution2.nodeCounts[key] ?: 0
        nodeCountDiversity += Math.abs(nodeCount1 - nodeCount2)
    }

    // Normalize node count diversity
    nodeCountDiversity /= (solution1.treeSize + solution2.treeSize)

    // Calculate avg fitness diversity
    var avgFitnessDiversity = 0.0

    // Gather all different avg fitness individual values key
    val avgFitnessKeys = mutableSetOf<String>()
    avgFitnessKeys.addAll(solution1.finalIndividualFitness.avgMatchResult.individualValues.keys)
    avgFitnessKeys.addAll(solution2.finalIndividualFitness.avgMatchResult.individualValues.keys)

    // Calculate avg fitness diversity
    for (key in avgFitnessKeys) {
        val avgFitness1 = solution1.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0
        val avgFitness2 = solution2.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0

        avgFitnessDiversity += Math.abs(avgFitness1 - avgFitness2)
    }

    // Normalize avg fitness diversity
    avgFitnessDiversity /= (Math.abs(sumIndividualValues(solution1.finalIndividualFitness.avgMatchResult, true)) + Math.abs(sumIndividualValues(solution2.finalIndividualFitness.avgMatchResult, true)))

    // Calculate final diversity
    return w1 * avgFitnessDiversity + w2 * nodeCountDiversity
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [ ]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd){
    for (selectedRun in selectedRunStart..selectedRunEnd){
        // Prepare data
        val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
        val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
        val avgGenIndividualRatings = mutableListOf<Double>()
        val bestGenIndividualsRatingAvgFitness = mutableListOf<Double>()

        val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
        val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
        val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

        prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualsRatingAvgFitness, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

        var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
        var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

        var lastPhaseName = "";
        val generations = mutableListOf<String>()
        for (genProgressData in multiRunProgressData.gensProgressData) {
            generations.add(genProgressData.generation.toString() + genProgressData.executionPhaseName.substring(genProgressData.executionPhaseName.length - 6, genProgressData.executionPhaseName.length))
        }

        // Display data
        val dfRating = dataFrameOf(
            "Generation" to generations + generations + generations,
            "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
            "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
        )

        val dfRatingAvgFitness = dataFrameOf(
            "Generation" to generations,
            "AvgFitness" to bestGenIndividualsRatingAvgFitness,
            "Individual" to List(generations.size) { "Average" }
        )

        val dfAvgFitness = dataFrameOf(
            "Generation" to generations + generations + generations,
            "AvgFitness" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
            "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
        )

        // The plot variable stores a plot with the temperature values for each city
        val linePlotRating =
            letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Rating (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val linePlotRatingAvgFitness =
            letsPlot(dfRatingAvgFitness.toMap()) { x = "Generation"; y = "AvgFitness"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Rating - Avg fitness (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val linePlotAvgFitness =
            letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "AvgFitness"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Avg fitness (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val plots = listOf(linePlotRating, linePlotRatingAvgFitness, linePlotAvgFitness)
        //val plots = listOf(linePlotRating, null, linePlotRatingAvgFitness, null, linePlotAvgFitness, null)
        gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()
        //gggrid(plots, ncol=2, widths = listOf(1,0)).show()
    }
}

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 1, 0)) // Individual at position

val maxValues = mutableMapOf<String, Double>()
maxValues["SectorExploration"] = 5.0
maxValues["PowerUp_Pickup_Health"] = 10.0
maxValues["PowerUp_Pickup_Ammo"] = 15.0
maxValues["PowerUp_Pickup_Shield"] = 5.0
maxValues["MissilesFired"] = 20.0
maxValues["MissilesFiredAccuracy"] = 50.0
maxValues["SurvivalBonus"] = 5.0
maxValues["OpponentTrackingBonus"] = 5.0
maxValues["OpponentDestroyedBonus"] = 500.0
maxValues["DamageTakenPenalty"] = 50.0
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, null, null, null, null, null)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

// Find all fitness keys
val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            System.out.println(key + ": 0.0")
            continue
        }
        dataset.add(Math.abs(individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!) / maxValues[key]!!)
        System.out.println(key + ": " + individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    System.out.println("/////////////////////////////")
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

# Selected Individual Details

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 1, 0)) // Individual at position
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, null, null, null, null, null)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

# Genetic Diversity Graph
Genetic diversity of the population based on the avg fitness and node counts

### Calculation of genetic diversity between 2 individuals
1. Difference in avg fitness -> df
2. Difference in node counts -> dn
3. Normalize both values and multiply by the weights -> diversity = w1 * df_norm +  w2 * dn_norm


In [ ]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
val w1 = 0.5
val w2 = 0.5
// Input params for visualization END

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

        prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, null, null, null, null, null, null)

        val gpAlgorithmRunGeneticDiversity = mutableListOf<List<Double?>>()

        for(i in 0 until bestGenIndividualsRating.size) {
            var gpAlgorithmGenGeneticDiversity = mutableListOf<Double?>()
            for (j in 0 until bestGenIndividualsRating.size) {
                if(j < i){
                    var diversity = calculateDiversity(bestGenIndividualsRating[i], bestGenIndividualsRating[j], w1, w2)
                    if(diversity > 1.0 || diversity < 0.0) {
                        throw Exception("Diversity out of bounds: " + diversity + ", i: " + i + ", j:" + j)
                    }
                    gpAlgorithmGenGeneticDiversity.add(diversity)
                }
                else {
                    gpAlgorithmGenGeneticDiversity.add(null)
                }
            }
            gpAlgorithmRunGeneticDiversity.add(gpAlgorithmGenGeneticDiversity)
        }

        var individuals = mutableListOf<Int>()
        val generations = mutableListOf<String>()

        for (i in 0 until gpAlgorithmRunGeneticDiversity.size) {
            generations.addAll(List(gpAlgorithmRunGeneticDiversity[i].size){(i + 1).toString()})

            for (i in 0 until gpAlgorithmRunGeneticDiversity[i].size) {
                individuals.add(i + 1)
            }
        }

        val df = dataFrameOf(
            "individuals" to individuals,
            "generations" to generations,
            "diversity" to gpAlgorithmRunGeneticDiversity.flatten()
        )

        val plot = letsPlot(df.toMap()) { x = "individuals"; y = "generations"; fill = "diversity" } + geomTile() + ggtitle("Genetic Diversity (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")
        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Master Tournament Graph
Tournament between generation best individuals

In [ ]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

val n = 3

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val Generation = mutableListOf<Int>()
        val Rating = mutableListOf<Double>()

        val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].masterTournamentGraphData;
        for (i in 0 until masterTournamentGraphData.size) {
            Generation.addAll(List(n) {i + 1})
            val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
            Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
        }

        val data = mapOf<String, Any>(
            "Generation" to Generation,
            "Rating" to Rating
        )

        val plot = letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Master Tournament (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Convergence graph
Best individual in the final generation is matched with every best individual from previous generations

In [ ]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val convergenceGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;

        var generations = mutableListOf<String>()
        var fitnesses = mutableListOf<Float>()

        var lastOpponentMatchIndex: Int = -1;
        for (individualMatch in convergenceGraphData.finalIndividualFitness.individualMatchResults){
            // If generations already contains individualMatch.opponentsIDs[0] then just sum fitness
            lastOpponentMatchIndex = generations.indexOf(individualMatch.opponentsIDs[0].toString());

            // Individual already played against this individual -> just sum fitness
            if(lastOpponentMatchIndex > -1){
                fitnesses[lastOpponentMatchIndex] += individualMatch.value;
            }
            else {
                generations.add(individualMatch.opponentsIDs[0].toString())
                fitnesses.add(individualMatch.value)
            }
        }

        // Display data
        val df = dataFrameOf(
            "Generation" to generations,
            "Fitness" to fitnesses
        )

        val plot = letsPlot(df.toMap()) { x = "Generation"; y = "Fitness"} + geomLine() + geomPoint(shape = 20) + ggtitle("Convergence Graph (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")
        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Grand Master Tournament
1. Master tournament between bestRunSolutions (final best gen solution from each run) -> For each method individually

In [ ]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 10
// Input params for visualization END

val n = 3

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    val IndividualIDs = mutableListOf<Int>()
    val Rating = mutableListOf<Double>()

    val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].masterMasterTournamentGraphData;
    for (i in 0 until masterTournamentGraphData.size) {
        IndividualIDs.addAll(List(n) { i })
        val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
        Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
    }

    val data = mapOf<String, Any>(
        "IndividualID" to IndividualIDs,
        "Rating" to Rating
    )

    val plot = letsPlot(data) { y = "IndividualID"; x = "Rating"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Master Tournament (Conf: " + selectedConfiguration + ")")

    plots.add(plot)
    plots.add(null)
}

gggrid(plots, ncol=2, widths = listOf(1.5, 0)).show()

# Final Master Tournament
1. Master tournament between bestRunSolutions (final best gen solution from each run - they each represent one individual on graph) for all methods.

In [ ]:
val n = 3

val IndividualIDs = mutableListOf<Long>()
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;
for (i in 0 until masterTournamentGraphData.size) {
    IndividualIDs.addAll(List(n){ masterTournamentGraphData[i].individualId })
    val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))

    Method.addAll(List(n){ masterTournamentGraphData[i].configurationName })
}

val data = dataFrameOf(
    "IndividualID" to IndividualIDs,
    "Rating" to Rating,
    "Method" to Method
)

val plot = letsPlot(data.toMap()) { y = "IndividualID"; x = "Rating"; color = "Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament") + ggtb()

plot.show()

# Final Master Tournament (Grouped) - Method 1

In [ ]:
val methodScores = mutableListOf<MethodScore>()

val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;

// Initialize method scores
for(config in gpAlgorithmExecutor.configuration.Configurations){
    methodScores.add(MethodScore(config.Name))
}

val calculator: TwoPlayerTrueSkillCalculator = TwoPlayerTrueSkillCalculator() // TODO Should we use TwoTeamTrueSkillCalculator instead?
val gameInfo: GameInfo = GameInfo.getDefaultGameInfo()

var playerIndexes = List(masterTournamentGraphData.size){ it }
playerIndexes = playerIndexes.shuffled()

for(i in 0 until masterTournamentGraphData.size){
    for(j in i until masterTournamentGraphData.size) {
        if(i == j) {
            continue
        }

        val individual1 = masterTournamentGraphData[i]
        val individual2 = masterTournamentGraphData[j]

        if(individual1.configurationName == individual2.configurationName){
            continue
        }

        val individual1MethodScore = methodScores.find { it.methodName == individual1.configurationName }!!
        val individual2MethodScore = methodScores.find { it.methodName == individual2.configurationName }!!

        val individual1Score = individual1.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual2.individualId.toInt() }.map { it.value }.sum()
        val individual2Score = individual2.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual1.individualId.toInt() }.map { it.value }.sum()

        // Determine match result & set scores
        var scores: List<Int>
        if(individual1Score < individual2Score){
            individual1MethodScore.wins++
            individual2MethodScore.losses++
            scores = listOf(1,2)
        }
        else if(individual1Score > individual2Score) {
            individual1MethodScore.losses++
            individual2MethodScore.wins++
            scores = listOf(2, 1)
        }
        else {
            individual1MethodScore.draws++
            individual2MethodScore.draws++
            scores = listOf(1,1)
        }

        // Update player ratings
        val team1 = Team(individual1MethodScore.player, TrueSkillRating(individual1MethodScore.player.freeForAllTrueSkill.rating, individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation))

        val team2 = Team(individual2MethodScore!!.player, TrueSkillRating(individual2MethodScore.player.freeForAllTrueSkill.rating, individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation))

        val teams = Team.concat(team1, team2)

        val newRatings = calculator.calculateNewRatings(gameInfo, teams, scores.get(0), scores.get(1))

        val player1NewRating = newRatings[individual1MethodScore.player]
        val player2NewRating = newRatings[individual2MethodScore.player]

        individual1MethodScore.player.freeForAllTrueSkill.rating =  player1NewRating!!.rating
        individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation =  player1NewRating.ratingDeviation

        individual2MethodScore.player.freeForAllTrueSkill.rating =  player2NewRating!!.rating
        individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation = player2NewRating.ratingDeviation
    }
}

val dataTemp = dataFrameOf(
    "Method" to methodScores.map { it.methodName },
    "Wins" to methodScores.map { it.wins },
    "Draws" to methodScores.map { it.draws },
    "Losses" to methodScores.map { it.losses }
)

System.out.println(dataTemp)

val n = 3
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

for(i in 0 until methodScores.size){
    Method.addAll(List(n){ methodScores.get(i).methodName })

    Rating.addAll(listOf(methodScores.get(i).player.freeForAllTrueSkill.rating - (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation), methodScores.get(i).player.freeForAllTrueSkill.rating, methodScores.get(i).player.freeForAllTrueSkill.rating + (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

plot.show()

# Final Master Tournament (Grouped) - Method 2

In [ ]:
val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;

var methodRatings = mutableListOf<MethodRating>()

// Initialize method raings
for(config in gpAlgorithmExecutor.configuration.Configurations){
    methodRatings.add(MethodRating(config.Name))
}

for(mmethodRating in methodRatings){
    // find all individuals from masterTournamentGraphData with the same method name
    val individuals = masterTournamentGraphData.filter { it.configurationName == mmethodRating.methodName }

    // Calculate group rating and std deviation
    var groupRating = 0.0
    var groupStdDeviation = 0.0

    for(individual in individuals){
        groupRating += individual.finalIndividualFitness.additionalValues["Rating"]!!
        groupStdDeviation += Math.pow(individual.finalIndividualFitness.additionalValues["StdDeviation"]!!, 2.0)
    }

    groupStdDeviation = Math.sqrt(groupStdDeviation)

    mmethodRating.rating = groupRating + 3 * groupStdDeviation
    mmethodRating.stdDeviation = groupStdDeviation
}

val n = 3
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

for(mmethodRating in methodRatings){
    Method.addAll(List(n){ mmethodRating.methodName })

    val rating = Math.abs(mmethodRating.rating)
    Rating.addAll(listOf(rating - (2 * mmethodRating.stdDeviation), rating, rating + (2 * mmethodRating.stdDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

plot.show()

# Final Master Tournament (Grouped) - Method 3
Only compare individuals from the same run

In [ ]:
val finalMasterTournamentsGraphData = progressData.finalMasterTournamentGraphData

val finalMasterTournamentGraphData = finalMasterTournamentsGraphData[3]

val playerGroups = mutableMapOf<String, MutableList<GPProgramSolutionSimple>>()

val methodScores = mutableListOf<MethodScore>()

val methodNamesIncludedInTheAnalysis = listOf("SwissSystem", "SET", "DET", "KRandomOpponents", "TrueSkill_Default_SSOS", "TrueSkill_Mean_SSOS", "TrueSkill_RatingUnchanged_SSOS", "TrueSkill_RatingAll_SSOS", "TrueSkill_Default_KRandomOpponents", "Glicko2", "Elo")

//val methodNamesIncludedInTheAnalysis = listOf("SwissSystem", "SET", "DET", "KRandomOpponents", "Glicko2", "Elo", "TrueSkill_Default_KRandomOpponents")

//val methodNamesIncludedInTheAnalysis = listOf("TrueSkill_Default_SwissSystem", "TrueSkill_Mean_SwissSystem", "TrueSkill_RatingUnchanged_SwissSystem", "TrueSkill_RatingAll_SwissSystem", "TrueSkill_Default_KRandomOpponents")

//val methodNamesIncludedInTheAnalysis = listOf("SwissSystem", "SET", "DET", "KRandomOpponents", "TrueSkill_Default_SwissSystem", "TrueSkill_Mean_SwissSystem", "TrueSkill_RatingUnchanged_SwissSystem", "TrueSkill_RatingAll_SwissSystem", "TrueSkill_Default_KRandomOpponents")

//val methodNamesIncludedInTheAnalysis = listOf("Glicko2", "Elo", "TrueSkill_Mean_SwissSystem")

//val methodNamesIncludedInTheAnalysis = listOf("SET", "SwissSystem")

// Initialize method scores
for(config in gpAlgorithmExecutor.configuration.Configurations){
    if(methodNamesIncludedInTheAnalysis.indexOf(config.Name) > -1) {
        methodScores.add(MethodScore(config.Name))
    }
}

for(individual in finalMasterTournamentGraphData){
    if(methodNamesIncludedInTheAnalysis.indexOf(individual.configurationName) > -1) {
        if (playerGroups.containsKey(individual.configurationName)) {
            playerGroups[individual.configurationName]?.add(individual)
        } else {
            playerGroups[individual.configurationName] = mutableListOf(individual)
        }
    }
}

val calculator: TwoPlayerTrueSkillCalculator = TwoPlayerTrueSkillCalculator() // TODO Should we use TwoTeamTrueSkillCalculator instead?
val gameInfo: GameInfo = GameInfo.getDefaultGameInfo()

var playerIndexes = List(playerGroups[methodScores.get(0).methodName]!!.size) {it}

val random = Random(123)
playerIndexes = playerIndexes.shuffled(random)

for(playerIndex in playerIndexes) {
    var playerGroupKeys1 = playerGroups.keys.toList()
    playerGroupKeys1 = playerGroupKeys1.shuffled(random)

    var playerGroupKeys2 = playerGroups.keys.toList()
    playerGroupKeys2 = playerGroupKeys2.shuffled(random)

    for(playerGroupKey1 in playerGroupKeys1) {
        for (playerGroupKey2 in playerGroupKeys2) {
            if (playerGroupKey1 == playerGroupKey2) {
                continue
            }

            val playerGroup1 = playerGroups[playerGroupKey1]!!
            val playerGroup2 = playerGroups[playerGroupKey2]!!

            val individual1 = playerGroup1[playerIndex]
            val individual2 = playerGroup2[playerIndex]

            // Get match results
            val individual1Score =
                individual1.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual2.individualId.toInt() }
                    .map { it.value }.sum()
            val individual2Score =
                individual2.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual1.individualId.toInt() }
                    .map { it.value }.sum()

            val individual1MethodScore = methodScores.find { it.methodName == individual1.configurationName }!!
            val individual2MethodScore = methodScores.find { it.methodName == individual2.configurationName }

            //System.out.println(individual1.configurationName + "(" + individual1.individualId + ") vs " + individual2.configurationName  + "(" + individual2.individualId + ") -> " + individual1Score + ": " + individual2Score)

            // Determine match result & set scores
            var scores: List<Int>
            if (individual1Score < individual2Score) {
                individual1MethodScore.wins++
                scores = listOf(1, 2)
            } else if (individual1Score > individual2Score) {
                individual1MethodScore.losses++
                scores = listOf(2, 1)
            } else {
                individual1MethodScore.draws++
                scores = listOf(1, 1)
            }

            // Update player ratings
            val team1 = Team(
                individual1MethodScore.player,
                TrueSkillRating(
                    individual1MethodScore.player.freeForAllTrueSkill.rating,
                    individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation
                )
            )

            val team2 = Team(
                individual2MethodScore!!.player,
                TrueSkillRating(
                    individual2MethodScore.player.freeForAllTrueSkill.rating,
                    individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation
                )
            )

            val teams = Team.concat(team1, team2)

            val newRatings = calculator.calculateNewRatings(gameInfo, teams, scores.get(0), scores.get(1))

            val player1NewRating = newRatings[individual1MethodScore.player]
            //val player2NewRating = newRatings[individual2MethodScore.player]

            individual1MethodScore.player.freeForAllTrueSkill.rating = player1NewRating!!.rating
            individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation = player1NewRating.ratingDeviation

            // Also set Glicko 2 rating
            if (individual1Score < individual2Score) {
                individual1MethodScore.player.addGame(Game(GameResult.WIN, individual1MethodScore.player, individual2MethodScore.player, "a"))
            } else if (individual1Score > individual2Score) {
                individual1MethodScore.player.addGame(Game(GameResult.LOSE, individual1MethodScore.player, individual2MethodScore.player, "a"))
            } else {
                individual1MethodScore.player.addGame(Game(GameResult.DRAW, individual1MethodScore.player, individual2MethodScore.player, "a"))
            }

            // Only update individual1MethodScore.player !!!
            //individual2MethodScore.player.freeForAllTrueSkill.rating =  player2NewRating!!.rating
            //individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation = player2NewRating.ratingDeviation
        }
    }
}

val players: HashMap<String, Player> = HashMap()

for(methodScore: MethodScore in methodScores){
    players[methodScore.player.id] = methodScore.player
}

Glicko2RatingCalculator.calculateNewRatings(players, false)

// Sort methodScores
methodScores.sortBy { it.player.glicko2Rating.rating }
//methodScores.sortBy { it.player.freeForAllTrueSkill.rating }

// Temp display
val dataTemp = dataFrameOf(
    "Method" to methodScores.map { it.methodName },
    "Wins" to methodScores.map { it.wins },
    "Draws" to methodScores.map { it.draws },
    "Losses" to methodScores.map { it.losses }
)

System.out.println(dataTemp)

val n = 3
val Rating = mutableListOf<Double>()
var Method = mutableListOf<String>()

for(i in 0 until methodScores.size){
    Method.addAll(List(n){ methodScores.get(i).methodName})

    for(j in 0 until Method.size)
    {
        if(Method[j].length > 20){
            Method[j] = Method[j].substring(0,20)
        }
    }

    Rating.addAll(listOf(methodScores.get(i).player.freeForAllTrueSkill.rating - (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation), methodScores.get(i).player.freeForAllTrueSkill.rating, methodScores.get(i).player.freeForAllTrueSkill.rating + (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

RatingIntervalPlot.displayChart(methodScores.map { it.player }.toCollection(ArrayList()), RatingType.GLICKO2, "Rating Interval")

plot.show()

# Final Master Tournament (Grouped) - Method 3 - Confidence bands

In [10]:
fun getFinalMasterTournamentGraphData(genNum : Int, methodNamesIncludedInTheAnalysis: List<String>): List<MethodScore>{
    val finalMasterTournamentsGraphData = progressData.finalMasterTournamentGraphData

    val finalMasterTournamentGraphData = finalMasterTournamentsGraphData[genNum]

    val playerGroups = mutableMapOf<String, MutableList<GPProgramSolutionSimple>>()

    val methodScores = mutableListOf<MethodScore>()

// Initialize method scores
    for(config in gpAlgorithmExecutor.configuration.Configurations){
        if(methodNamesIncludedInTheAnalysis.indexOf(config.Name) > -1) {
            methodScores.add(MethodScore(config.Name))
        }
    }

    for(individual in finalMasterTournamentGraphData){
        if(methodNamesIncludedInTheAnalysis.indexOf(individual.configurationName) > -1) {
            if (playerGroups.containsKey(individual.configurationName)) {
                playerGroups[individual.configurationName]?.add(individual)
            } else {
                playerGroups[individual.configurationName] = mutableListOf(individual)
            }
        }
    }

    var playerIndexes = List(playerGroups[methodScores.get(0).methodName]!!.size) {it}

    val random = Random(123)
    playerIndexes = playerIndexes.shuffled(random)

    for(playerIndex in playerIndexes) {
        var playerGroupKeys1 = playerGroups.keys.toList()
        playerGroupKeys1 = playerGroupKeys1.shuffled(random)

        var playerGroupKeys2 = playerGroups.keys.toList()
        playerGroupKeys2 = playerGroupKeys2.shuffled(random)

        for(playerGroupKey1 in playerGroupKeys1) {
            for (playerGroupKey2 in playerGroupKeys2) {
                if (playerGroupKey1 == playerGroupKey2) {
                    continue
                }

                val playerGroup1 = playerGroups[playerGroupKey1]!!
                val playerGroup2 = playerGroups[playerGroupKey2]!!

                val individual1 = playerGroup1[playerIndex]
                val individual2 = playerGroup2[playerIndex]

                // Get match results
                val individual1Score =
                    individual1.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual2.individualId.toInt() }
                        .map { it.value }.sum()
                val individual2Score =
                    individual2.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual1.individualId.toInt() }
                        .map { it.value }.sum()

                val individual1MethodScore = methodScores.find { it.methodName == individual1.configurationName }!!
                val individual2MethodScore = methodScores.find { it.methodName == individual2.configurationName }

                //System.out.println(individual1.configurationName + "(" + individual1.individualId + ") vs " + individual2.configurationName  + "(" + individual2.individualId + ") -> " + individual1Score + ": " + individual2Score)

                // Determine match result & set scores
                if (individual1Score < individual2Score) {
                    individual1MethodScore.wins++
                } else if (individual1Score > individual2Score) {
                    individual1MethodScore.losses++
                } else {
                    individual1MethodScore.draws++
                }

                // Also set Glicko 2 rating
                if (individual1Score < individual2Score) {
                    individual1MethodScore.player.addGame(Game(GameResult.WIN, individual1MethodScore.player, individual2MethodScore?.player, "a"))
                } else if (individual1Score > individual2Score) {
                    individual1MethodScore.player.addGame(Game(GameResult.LOSE, individual1MethodScore.player, individual2MethodScore?.player, "a"))
                } else {
                    individual1MethodScore.player.addGame(Game(GameResult.DRAW, individual1MethodScore.player, individual2MethodScore?.player, "a"))
                }
            }
        }
    }

    val players: HashMap<String, Player> = HashMap()

    for(methodScore: MethodScore in methodScores){
        players[methodScore.player.id] = methodScore.player
    }

    Glicko2RatingCalculator.calculateNewRatings(players, false)

    // Sort methodScores
    methodScores.sortBy { it.player.glicko2Rating.rating }

    return methodScores
}

In [11]:
class FinalMethodRating(val methodName: String) {
    var ratings: MutableList<Double> = mutableListOf<Double>()
    var ratingDeviations: MutableList<Double> = mutableListOf<Double>()
    var ratingIntervalLowers: MutableList<Double> = mutableListOf<Double>()
    var ratingIntervalUppers: MutableList<Double> = mutableListOf<Double>()
}

In [13]:
import org.jetbrains.letsPlot.commons.values.Color

val selectedMethods = listOf("SwissSystem", "SET", "DET", "KRandomOpponents", "TrueSkill_Default_SSOS", "TrueSkill_Mean_SSOS", "TrueSkill_RatingUnchanged_SSOS", "TrueSkill_RatingAll_SSOS", "TrueSkill_Default_KRandomOpponents", "Glicko2", "Elo")

//val selectedMethods = listOf("TrueSkill_Default_SSOS", "TrueSkill_Default_KRandomOpponents", "Glicko2", "Elo")

//val selectedMethods = listOf("TrueSkill_Default_SSOS", "TrueSkill_Default_KRandomOpponents", "SwissSystem", "SET", "DET", "KRandomOpponents", "Glicko2", "Elo")

//val selectedMethods = listOf("TrueSkill_Mean_SSOS", "Glicko2", "Elo")

val finalMasterTournamentsGraphData = progressData.finalMasterTournamentGraphData
val methodScoresGen0 = getFinalMasterTournamentGraphData(0, selectedMethods)
//val numAlgorithms = methodScoresGen0.size
//val xValues = listOf(1400,2828,4284,5768,7280,8820,10388,11984,13608,15260,16940,18648,20384) // x-axis values for generations
//val xValues = listOf(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15) // x-axis values for generations
val xValues = listOf(1,3,5,6) // x-axis values for generations

var plot = letsPlot(mapOf("gen" to xValues))

val methodRatings = mutableListOf<FinalMethodRating>()

for(methodScore in methodScoresGen0) {
    // Initialize method ratings
    methodRatings.add(FinalMethodRating(methodScore.methodName))
}

//for loop foreach finalMasterTournamentsGraphData
for (genNum in 0 until finalMasterTournamentsGraphData.size) {
    val methodScores = getFinalMasterTournamentGraphData(genNum, selectedMethods)

    for(methodScore in methodScores){
        val methodRating = methodRatings.find { it.methodName == methodScore.methodName }!!

        methodRating.ratings.add(methodScore.player.getGlicko2Rating().rating)
        methodRating.ratingDeviations.add(methodScore.player.getGlicko2Rating().ratingDeviation)
        methodRating.ratingIntervalLowers.add(methodScore.player.getGlicko2Rating().getRatingIntervalLower())
        methodRating.ratingIntervalUppers.add(methodScore.player.getGlicko2Rating().getRatingIntervalUpper())
    }
}

// Create the plot with predefined colors for each method
var colors : Map<String, String> = mapOf(
    "KRandomOpponents" to "#f24738",
    "TrueSkill_Default_KRandomOpponents" to "#487ed4",
    "TrueSkill_RatingAll_SSOS" to "#48d462",
    "TrueSkill_RatingUnchanged_SSOS" to "#9c48d4",
    "TrueSkill_Default_SSOS" to "#fca138",
    "TrueSkill_Mean_SSOS" to "#f7fa55",
    "Glicko2" to "#a1560b",
    "Elo" to "#d448cd",
    "SwissSystem" to "#878386",
    "DET" to "#48d4cf",
    "SET" to "#d448aa"
)

for(methodRating in methodRatings){
    val yValues = methodRating.ratings

    // calculate the lower and upper bounds for the rating deviations
    val lowerBounds = methodRating.ratingIntervalLowers
    val upperBounds = methodRating.ratingIntervalUppers

    // prepare data for the current algorithm
    val data = mapOf(
            "Fitness Evaluations" to xValues,
            "Rating" to yValues,
            "lowerBounds" to lowerBounds,
            "upperBounds" to upperBounds,
            "Algorithm" to List(yValues.size) { methodRating.methodName }  // label each algorithm
    )

    // add points at the same locations as the line
    plot += geomPoint(data = data) {
        x = "Fitness Evaluations"
        y = "Rating"
        color = "Algorithm"
        shape = "Algorithm"
    }

    // add rating line to the plot (cycling through predefined colors)
    plot += geomLine(data = data) {
        x = "Fitness Evaluations"
        y = "Rating"
        color = "Algorithm"
        linetype = "Algorithm"
    }

    // add rating deviation ribbon to the plot (using the same color with transparency)
    plot += geomRibbon(data = data, alpha = 0.1, linetype = "blank", showLegend = false) {
        x = "Fitness Evaluations"
        ymin = "lowerBounds"
        ymax = "upperBounds"
        color = "Algorithm"
        fill = "Algorithm"
        linetype = "Algorithm"
    }
}

val minY = 1100
val maxY = 2000
//plot += scaleYContinuous(limits = minY to maxY)

val minX = xValues.minOrNull() ?: 0
val maxX = xValues.maxOrNull() ?: 0
plot += scaleXContinuous(
    breaks = xValues,
    //labels = xValues.take(valuesToShow).map { it.toInt().toString() }, // specify the tick marks
    limits = minX to maxX
)

plot += scaleColorManual(values = colors)
plot += scaleFillManual(values = colors)

/*xValues.forEach { evalPoint ->
    plot += geomVLine(xintercept = evalPoint, color = "#DFDFDF", linetype = "dashed") {
        // you can customize the appearance here
    }
}*/

val title = "CCEM ratings during optimisation"
val width = 2000
val height = 750

plot += ggsize(width, height)
plot += ggtitle(title)
plot += ggtb()  // enable Zoom and Pan interactivity

plot.show()

# Visualization test

In [ ]:
class EvalsFitnesses(val methodName: String) {
    var evals: MutableList<Int> = mutableListOf<Int>()
    var fitnesses: MutableList<Double> = mutableListOf<Double>()
}

In [ ]:
// Pop sizes for each generation (because HoF is used)
val pop_sizes = listOf(100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139)

In [ ]:
val method_set = EvalsFitnesses("SET")
method_set.evals = mutableListOf(198, 398, 600, 804, 1010, 1218, 1428, 1640, 1854, 2070, 2288, 2508, 2730, 2954, 3180, 3408, 3638, 3870, 4104, 4340, 4578, 4818, 5060, 5304, 5550, 5798, 6048, 6300, 6554, 6810, 7068, 7328, 7590, 7854, 8120, 8388, 8658, 8930, 9204, 9480, 9758, 10038, 10320, 10604, 10890, 11178, 11468, 11760, 12054, 12350, 12648, 12948, 13250, 13554, 13860, 14168, 14478, 14790, 15104, 15420, 15738, 16058, 16380, 16704, 17030, 17358, 17688, 18020, 18354, 18690, 19028, 19368, 19710, 20054, 20400, 20748, 21098, 21450, 21804, 22160, 22518, 22878, 23240, 23604, 23970, 24338, 24708, 25080, 25454, 25830, 26208, 26588, 26970, 27354, 27740, 28128, 28518, 28910, 29304, 29700, 30098, 30498, 30900, 31304, 31710, 32118, 32528, 32940, 33354, 33770, 34188, 34608, 35030)
method_set.fitnesses = mutableListOf(0.439903447, 0.731414562, 0.186657629, 0.530790866, 0.138404214, 0.176979275, 0.239382579, 0.816853543, 0.869478873, 0.630493356, 0.016121192, 0.189600227, 0.641260038, 0.025961543, 0.586439577, 0.753810242, 0.180023532, 0.488073984, 0.308321331, 0.638504306, 0.490158571, 0.258799084, 0.621709791, 0.509170079, 0.086094143, 0.485590143, 0.13184647, 0.228755672, 0.832012682, 0.701089059, 0.328370587, 0.342129899, 0.351101795, 0.265162769, 0.180693008, 0.256607341, 0.951378964, 0.283455415, 0.171231906, 0.920307575, 0.681635567, 0.864087176, 0.239024502, 0.193405262, 0.726647686, 0.25907648, 0.4285142, 0.986623937, 0.030353922, 0.437521767, 0.207613479, 0.084313719, 0.473143847, 0.562343522, 0.910286964, 0.224038301, 0.482671974, 0.351758193, 0.782456447, 0.384351043, 0.976682225, 0.678190302, 0.974743173, 0.897263294, 0.149593243, 0.877365142, 0.011558603, 0.1557312, 0.887790685, 0.669267852, 0.487437895, 0.556901317, 0.581374951, 0.856456284, 0.671055906, 0.197190315, 0.204940751, 0.147418842, 0.221109926, 0.652649469, 0.97314538, 0.636046244, 0.15949072, 0.898561395, 0.82668804, 0.408610912, 0.5841202, 0.074307902, 0.932712765, 0.129644038, 0.37590348, 0.767055955, 0.099928315, 0.895598117, 0.056485151, 0.325012493, 0.174784623, 0.303692495, 0.931053468, 0.593888113, 0.040326494, 0.105331118, 0.700299731, 0.473459434, 0.071032296, 0.336672125, 0.172060431, 0.344641175, 0.858699577, 0.411634846, 0.996749124, 0.806376904, 0.466053417)

In [ ]:
val method_det = EvalsFitnesses("DET")
method_det.evals = mutableListOf(396, 796, 1200, 1608, 2020, 2436, 2856, 3280, 3708, 4140, 4576, 5016, 5460, 5908, 6360, 6816, 7276, 7740, 8208, 8680, 9156, 9636, 10120, 10608, 11100, 11596, 12096, 12600, 13108, 13620, 14136, 14656, 15180, 15708, 16240, 16776, 17316, 17860, 18408, 18960, 19516, 20076, 20640, 21208, 21780, 22356, 22936, 23520, 24108, 24700, 25296, 25896, 26500, 27108, 27720, 28336, 28956, 29580, 30208, 30840, 31476, 32116, 32760, 33408, 34060, 34716)
method_det.fitnesses = mutableListOf(0.547491521, 0.5597684, 0.49511508, 0.786840726, 0.5207122, 0.93172234, 0.289128145, 0.091372053, 0.367921134, 0.144054845, 0.640034715, 0.954036306, 0.879040875, 0.793333657, 0.211058885, 0.313550043, 0.544038582, 0.056937188, 0.538266265, 0.659227055, 0.486215423, 0.184087018, 0.363172087, 0.523569888, 0.665710188, 0.826184176, 0.326735941, 0.266694006, 0.060286621, 0.205893908, 0.802595536, 0.665956115, 0.998656725, 0.757638042, 0.081988341, 0.09530958, 0.337239872, 0.302322398, 0.633065309, 0.947944269, 0.753764487, 0.515880531, 0.030247354, 0.061473881, 0.887447113, 0.869095918, 0.978023495, 0.768696752, 0.180971126, 0.802529014, 0.493825119, 0.123321006, 0.864563983, 0.465275724, 0.286880888, 0.359235969, 0.956353379, 0.509354139, 0.361029698, 0.670381737, 0.324292748, 0.667695709, 0.234918144, 0.518482411, 0.183331536, 0.081197797)

In [ ]:
val method_kRandom = EvalsFitnesses("KRandomOpponents")
method_kRandom.evals = mutableListOf(700, 1406, 2120, 2840, 3568, 4302, 5044, 5792, 6548, 7310, 8080, 8856, 9640, 10430, 11228, 12032, 12844, 13662, 14488, 15320, 16160, 17006, 17860, 18720, 19588, 20462, 21344, 22232, 23128, 24030, 24940, 25856, 26780, 27710, 28648, 29592, 30544, 31502, 32468, 33440)
method_kRandom.fitnesses = mutableListOf(0.69767464, 0.391458431, 0.532892121, 0.645194153, 0.38353796, 0.815195072, 0.782790669, 0.503980705, 0.606635802, 0.913637584, 0.534088786, 0.313823134, 0.38507865, 0.018389821, 0.661127537, 0.798529582, 0.142016712, 0.698183682, 0.45342242, 0.716003136, 0.309384578, 0.313429024, 0.122222572, 0.717395337, 0.249916511, 0.344825561, 0.529965398, 0.125037262, 0.599317846, 0.553261004, 0.452547955, 0.517720813, 0.811373258, 0.07322712, 0.047815457, 0.325665541, 0.046101035, 0.52064111, 0.219510897, 0.616171223)

In [ ]:
val method_swissSystem = EvalsFitnesses("SwissSystem")
method_swissSystem.evals = mutableListOf(700, 1414, 2128, 2856, 3584, 4326, 5068, 5824, 6580, 7350, 8120, 8904, 9688, 10486, 11284, 12096, 12908, 13734, 14560, 15400, 16240, 17094, 17948, 18816, 19684, 20566, 21448, 22344, 23240, 24280, 25320, 26376, 27432, 28504, 29576, 30664, 31752, 32856, 33960, 35080)
method_swissSystem.fitnesses = mutableListOf(0.530111756, 0.661548876, 0.003183249, 0.720143272, 0.314845348, 0.079286246, 0.338426468, 0.350876833, 0.868091476, 0.827195338, 0.532509021, 0.654011672, 0.366759851, 0.105330891, 0.158682784, 0.336486454, 0.213629796, 0.587275367, 0.719965645, 0.240351923, 0.955888235, 0.048398563, 0.836926573, 0.479720947, 0.121932748, 0.289664408, 0.875107811, 0.507661039, 0.892866058, 0.09202479, 0.025413175, 0.320393752, 0.606796714, 0.386810503, 0.241900173, 0.693111295, 0.840296917, 0.447282142, 0.311098566, 0.088724206)

In [ ]:
val min_evals = 198
val max_evals = 35080

// Create a data frame for plotting (Line Chart)
val data = dataFrameOf(
    "evals" to method_set.evals + method_det.evals + method_kRandom.evals + method_swissSystem.evals,
    "fitnesses" to method_set.fitnesses + method_det.fitnesses + method_kRandom.fitnesses + method_swissSystem.fitnesses,
    "method" to List(method_set.evals.size) { method_set.methodName } +
            List(method_det.evals.size) { method_det.methodName } +
            List(method_kRandom.evals.size) { method_kRandom.methodName } +
            List(method_swissSystem.evals.size) { method_swissSystem.methodName }
)
var plot = letsPlot(data.toMap()) { x = "evals"; y = "fitnesses"; color = "method" } +
    geomLine() {linetype = "method"} +
    geomPoint() {shape = "method"} +
    scaleXContinuous(limits = min_evals to max_evals) +
    scaleYContinuous(limits = 0.0 to 1.0) +
    ggtitle("Fitnesses vs Evals for different methods") +
    ggsize(1200, 600) +
    ggtb()

// Add vertical lines every 1000 evals
val verticalLines = listOf<Int>(700, 1414, 2128, 2856, 3584, 4326, 5068, 5824, 6580, 7350, 8120, 8904, 9688, 10486, 11284, 12096, 12908, 13734, 14560, 15400, 16240, 17094, 17948, 18816, 19684, 20566, 21448, 22344, 23240, 24280, 25320, 26376, 27432, 28504, 29576, 30664, 31752, 32856, 33960, 35080)

verticalLines.forEach { evalPoint ->
    plot += geomVLine(xintercept = evalPoint, color = "grey", linetype = "dashed") {
        // you can customize the appearance here
    }
}

plot.show()

In [ ]:
import java.awt.Color

var pop_size = listOf(10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60)

var SET_required_matches = listOf(18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118)

var DET_required_matches = listOf(36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124, 128, 132, 136, 140, 144, 148, 152, 156, 160, 164, 168, 172, 176, 180, 184, 188, 192, 196, 200, 204, 208, 212, 216, 220, 224, 228, 232, 236)

var KRandom_required_matches = listOf(40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124, 128, 132, 136, 140, 144, 148, 152, 156, 160, 164, 168, 172, 176, 180, 184, 188, 192, 196, 200, 204, 208, 212, 216, 220, 224, 228, 232, 236, 240)

var SwissSystem_required_matches = listOf(40, 48, 48, 56, 56, 64, 64, 90, 90, 100, 100, 110, 110, 120, 120, 130, 130, 140, 140, 150, 150, 160, 160, 204, 204, 216, 216, 228, 228, 240, 240, 252, 252, 264, 264, 276, 276, 288, 288, 300, 300, 312, 312, 324, 324, 336, 336, 348, 348, 360, 360)

var SSOS_required_matches = listOf(40, 48, 48, 56, 56, 64, 64, 90, 90, 100, 100, 110, 110, 120, 120, 130, 130, 140, 140, 150, 150, 160, 160, 204, 204, 216, 216, 228, 228, 240, 240, 252, 252, 264, 264, 276, 276, 288, 288, 300, 300, 312, 312, 324, 324, 336, 336, 348, 348, 360, 360)

var round_robin_required_matches = listOf(45, 55, 66, 78, 91, 105, 120, 136, 153, 171, 190, 210, 231, 253, 276, 300, 325, 351, 378, 406, 435, 465, 496, 528, 561, 595, 630, 666, 703, 741, 780, 820, 861, 903, 946, 990, 1035, 1081, 1128, 1176, 1225, 1275, 1326, 1378, 1431, 1485, 1540, 1596, 1653, 1711, 1770)

// Create a data frame for plotting (Line Chart) where x is the population size and y is the required matches and line color is the method (color needs to be predefined for each method)
val data_pop = dataFrameOf(
    "Population size" to pop_size + pop_size + pop_size + pop_size + pop_size + pop_size,
    "Required matches" to SET_required_matches + DET_required_matches + KRandom_required_matches + SwissSystem_required_matches + SSOS_required_matches + round_robin_required_matches,
    "Opponent Selection Strategy" to List(SET_required_matches.size) { "SET" } +
            List(DET_required_matches.size) { "DET" } +
            List(KRandom_required_matches.size) { "KRandomOpponents" } +
            List(SwissSystem_required_matches.size) { "SwissSystem" } +
            List(SSOS_required_matches.size) { "SSOS" } +
            List(round_robin_required_matches.size) { "RoundRobin" }
)

// Create the plot with predefined colors for each method
var colors : Map<String, Color> = mapOf(
    "SET" to Color.BLUE,
    "DET" to Color.RED,
    "KRandomOpponents" to Color.GRAY,
    "SwissSystem" to Color.ORANGE,
    "SSOS" to Color.GRAY,
    "RoundRobin" to Color.BLACK
)

var plot_pop = letsPlot(data_pop.toMap()) { x = "Population size"; y = "Required matches"; color = "Opponent Selection Strategy" } +
        geomLine() {linetype = "Opponent Selection Strategy"} +
        //geomPoint() {shape = "method"} +
        scaleXContinuous(limits = 10 to 60) +
        scaleYContinuous(limits = 0 to 1800) +
        scaleColorManual(values = colors) +
        ggtitle("Required matches vs Population size for different methods") +
        ggsize(600, 600) +
        ggtb() // enable Zoom and Pan interactivity

plot_pop